In [ ]:
from fastrenewables.tabular.learner import *
from fastrenewables.tabular.data import *
from fastrenewables.tabular.core import *
from fastrenewables.metrics import crps_for_quantiles
import pandas as pd
import zipfile
import zipfile, re, os
import numpy as np
from fastai.torch_basics import *
from fastai.metrics import rmse, mae
from fastai.tabular.all import *
import seaborn as sns
import properscoring as ps
import dcor


In [ ]:
# !pip install fastdownload
# pip install dcor
# pip install properscoring

In [ ]:
# https://github.com/fastai/fastdownload/issues/16
# pip install "git+https://github.com/GenevieveBuckley/fastdownload/@fmod-exists-error"

In [ ]:
from fastdownload import FastDownload

In [ ]:
d = FastDownload()
path = d.get('https://www.dropbox.com/s/pqenrr2mcvl0hk9/GEFCom2014.zip?dl=1')

In [ ]:
def read_single_file(file_name):
    df = pd.read_hdf(file_name, key="powerdata")
    return df

In [ ]:
def extract_single_file(path_to_zip_file, directory_to_extract_to):
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(path=directory_to_extract_to)

In [ ]:
def extract_zip_in_folder(toFolder):
    for root, dirs, files in os.walk(toFolder):
        for filename in files:
            if re.search(r'\.zip$', filename):
                fileSpec = os.path.join(root, filename)
                extract_single_file(fileSpec, root)

In [ ]:
def get_wind_speed(x, y):
    z = np.sqrt(x ** 2 + y ** 2)
    return z

def get_wind_direction(x, y):
    z = get_wind_speed(x, y)
    phi = 2 * np.arctan(y / (x + z + 1e-16))
    return phi

In [ ]:
def read_csv(file_name):
    df = pd.read_csv(file_name, sep=",")
    df.TIMESTAMP = pd.to_datetime(df.TIMESTAMP, infer_datetime_format=True, utc=True)
    df = df.rename(columns={"TIMESTAMP": "TimeUTC", "TARGETVAR": "PowerGeneration", "ZONEID":"TaskID"})
    df.set_index("TimeUTC", inplace=True)
    
    return df

In [ ]:
def create_complete_task(file_name_task, file_name_solution=None):
    df = read_csv(file_name_task)
    
    cols = [("U10", "V10"),("U100", "V100")]
        
    for c in cols:
        ws = get_wind_speed(df[c[0]].values, df[c[1]].values)
        wd = get_wind_direction(df[c[0]].values, df[c[1]].values)
        w_height = "100" if "100" in c[0] else "10"
        
        df[f"WindSpeed{w_height}m"] = ws
        df[f"SinWindDirection{w_height}m"] = np.sin(wd)
        df[f"CosWindDirection{w_height}m"] = np.cos(wd)   
    
    df["WindSpeed10m_t_m1"] = df.WindSpeed10m.shift(1).fillna(method='bfill')
#     df["WindSpeed10m_t_p1"] = df.WindSpeed10m.shift(-1).fillna(method='ffill')
    
    df["WindSpeed100m_t_m1"] = df.WindSpeed100m.shift(1).fillna(method='bfill')
#     df["WindSpeed100m_t_p1"] = df.WindSpeed100m.shift(-1).fillna(method='ffill')
    
    if file_name_solution is not None:
        df_solution = read_csv(file_name_solution)
        df_solution = df_solution[df_solution.TaskID==df.TaskID[0]]
        
#         check if timestamps match
        if (df.index == df_solution.index).sum() == df.shape[0]:
            
            df["PowerGeneration"] = df_solution.PowerGeneration.values
        else:
            raise Warning("Timestamps do not match.")
    df.dropna(inplace=True)
    return df  

In [ ]:
def merge_dfs(first_list, second_list):
    dfs = []
    if len(first_list)!= len(second_list):
        raise ValueError("Different lenghts of list.")
        
    for idx in range(len(first_list)):
        df1, df2 = first_list[idx], second_list[idx]
        if df1.TaskID[0] != df2.TaskID[0]:
            raise ValueError("Not sorted correctly. Not matching task ids.")
        dfs.append(pd.concat([df1, df2], axis=0))
        
    return dfs

In [ ]:
def prepare_for_tasks(dfs, gefcom_task=1):
    
    start_date = pd.to_datetime("2012-10-01", utc=True) + pd.DateOffset(months=gefcom_task-1)
    end_date = last_day_of_month(start_date)
    
    dfs_train = []
    for df in dfs:
        mask = df.index < start_date
        dfs_train.append(df[mask])
        
    dfs_test = []
    for df in dfs:
        mask = (df.index >= start_date) & (df.index <= end_date)
        dfs_test.append(df[mask])

    return pd.concat(dfs_train, axis=0), pd.concat(dfs_test, axis=0)

In [ ]:
import datetime

def last_day_of_month(any_day):
    # this will never fail
    # get close to the end of the month for any day, and add 4 days 'over'
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    # subtract the number of remaining 'overage' days to get last day of current month, or said programattically said, the previous day of the first of next month
    return next_month - datetime.timedelta(days=next_month.day)

In [ ]:
# uncomment if file is not yet downloaded

# !wget "https://www.dropbox.com/s/pqenrr2mcvl0hk9/GEFCom2014.zip"
# extract_single_file("GEFCom2014.zip", "./")
# extract_single_file("GEFCom2014 Data/GEFCom2014-W_V2.zip", "./")
# extract_zip_in_folder("./Wind/")

In [ ]:
ls

In [ ]:
files_task = !ls {"./Wind/Task\ 15/Task15_W_Zone1_10/*csv"}

In [ ]:
dfs_task = [create_complete_task(f) for f in files_task]
len(dfs_task)

In [ ]:
files_task_solution = !ls {"./Wind/Task\ 15/TaskExpVars15_W_Zone1_10/*csv"}
dfs_task_solution = [create_complete_task(f, './Wind/Solution to Task 15/solution15_W.csv') for f in files_task_solution]

In [ ]:
dfs = merge_dfs(dfs_task, dfs_task_solution)

In [ ]:
GEFCOM_TASK = 3

In [ ]:
df_train, df_test = prepare_for_tasks(dfs, gefcom_task=GEFCOM_TASK)

In [ ]:
df_train.head(2)

In [ ]:
df_train.tail(2)

In [ ]:
df_train.index.month

In [ ]:
df_train.dtypes

In [ ]:
df_train.TaskID.unique()

In [ ]:
cat_names = ["TaskID"]

cont_names = ['U10', 'V10', 'U100', 'V100',
       'WindSpeed10m', 'SinWindDirection10m', 'CosWindDirection10m',
       'WindSpeed100m', 'SinWindDirection100m', 'CosWindDirection100m',
       'WindSpeed10m_t_m1', 
#         'WindSpeed10m_t_p1', 
       'WindSpeed100m_t_m1',
#        'WindSpeed100m_t_p1'
             ]
y_names = ["PowerGeneration"]


In [ ]:
dls = RenewableDataLoaders.from_df(df_train, 
                                   cat_names=cat_names, 
                                   cont_names=cont_names, 
                                   y_names=y_names,
                                   bs=24*30, 
                                   shuffle=True
                                  )

In [ ]:
dls.dataset.items.WindSpeed10m.describe()

In [ ]:
dls.train_ds.items.head(2)

Principle Idea: Use a Bayesian neural network to forecast the expected power generation. Use the power generation from t-1 as input to the model during training. Also we should forecast the mean as well as the variance, essentially we train a one step ahead model.

To create a scenario forecast, we need to adapt the inference. Essentially we use the one step ahead model model to lets say the next 72 hours. Therefore, we replace the historical power measurements through the power forecast from the previous time stamp.

In [ ]:
quantiles = np.arange(0.01,1,0.01)
# quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]

In [ ]:
n_out = len(quantiles)
# n_out=1
learn = renewable_learner(dls, 
                          layers=[len(cont_names), 200, 300, 400,  n_out],
#                           metrics=[rmse, mae], 
                           n_out=n_out, 
                          loss_func=Quantile_Score(taus=list(quantiles))
                         )

In [ ]:
learn.model

In [ ]:
learn.lr_find()
# plt.show()

In [ ]:
plt.show()

In [ ]:
lr=1e-3

In [ ]:
learn.fit_one_cycle(5, lr_max=lr)

In [ ]:
learn.fit(10, lr=lr)

In [ ]:
# learn.fit(10, lr=lr)

In [ ]:
to_test = dls.train_ds.new(df_test)
to_test.setup()
to_test.items.WindSpeed100m.describe()

In [ ]:
dl_test = to_test.dataloaders(shuffle=False, drop_last=False)

In [ ]:
to_test.items.shape

In [ ]:
to_test.items.head(10)

In [ ]:
to_test.items.index.unique()

In [ ]:
yhat = learn.model(torch.tensor(to_test.cats.values).long().to("cuda:0"), torch.tensor(to_test.conts.values).float().to("cuda:0"))
yhat = to_np(yhat)

In [ ]:
yhat.shape

In [ ]:
# sort to avoid quantile crissing
yhat = np.sort(yhat, axis=1)

In [ ]:
to_test.items

In [ ]:
y = to_test.ys.values

In [ ]:
y.shape

In [ ]:
plt.figure(figsize=(16,9))
start = 50
end = start+24*3
plt.plot(y[start:end], label="y")
for q_i in range(0, yhat.shape[1], 3):
    plt.plot(yhat[start:end, q_i], alpha=0.1, c="black")
    
plt.plot(yhat[start:end, len(quantiles)//2], alpha=0.75, c="red", label="yhat_0.5")
# plt.ylim(-1,2)
plt.legend()
plt.show()

In [ ]:
yhat.shape

In [ ]:
crps_mean = ps.crps_ensemble(y.reshape(-1), yhat).mean()

In [ ]:
crps_mean

In [ ]:
crps_mean, crps_all = crps_for_quantiles(yhat, y.reshape(-1), quantiles=quantiles)
crps_mean

In [ ]:
start_id, end_id = 0,0
crps_values = []
for k,df in to_test.items.groupby("TaskID"):
    start_id = end_id
    end_id = start_id + df.shape[0]
    crps_mean, _ = crps_for_quantiles(yhat[start_id:end_id,:], y[start_id:end_id,:].reshape(-1), quantiles=quantiles)
    crps_values.append(crps_mean)

In [ ]:
sns.boxplot(crps_values)
plt.title("CRPS values of all tasks.")
plt.show()

In [ ]:
# dcor.energy_distance(y.reshape(-1), yhat)